In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install folium matplotlib
import folium
!pip install geopy
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

In [3]:
m = folium.Map(zoom_start = 6)

existing_dcs = [
    {'latitude': 35.393528, 'longitude': - 119.043732, 'name': 'Bakersfield, CA'},
    {'latitude': 38.2805, 'longitude': -104.4672, 'name': 'Pueblo, CO'},
    {'latitude': 44.979595, 'longitude': -93.276566, 'name': 'Woodbury, MN'},
    {'latitude': 40.028900, 'longitude': -77.590698, 'name': 'Chambersberg, PN'}
]

for location in existing_dcs:
    new_marker = folium.Marker(
        location=[location['latitude'], location['longitude']],
        popup=location['name'],
        icon=folium.Icon(color='red')
    )

    new_marker.add_to(m)

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Target Upcurve/Project/DC_Planning.csv')

In [6]:
df['latitude'] = df['geographic_loc'].str.extract(r'\((.*?),')[0].astype(float)
df['longitude'] = df['geographic_loc'].str.extract(r',(.*)\)', expand=False).astype(float)
df.head(5)

,guest_id,geographic_loc,order_frequency,avg_order_value,Electronics,Clothing,Home Appliances,Books,Beauty,returns_rate,guest_segmentation,shipping_preferences,promotional_response,state,country,latitude,longitude
0,G50029,"(41.9345228822601,-114.793454289252)",12.528133,9.092696,1,0,0,0,1,0.091774,Lapsed,Express,Medium,Nevada,United States,41.934523,-114.793454
1,G39532,"(28.4114060271366,-98.0530295882411)",17.571661,18.851202,1,1,0,0,0,0.300096,Repeat,Express,Low,Texas,United States,28.411406,-98.053030
2,G93168,"(37.8619825697659,-79.3247968858362)",9.699606,47.468598,1,1,0,0,1,0.580670,Lapsed,Express,Low,Virginia,United States,37.861983,-79.324797
3,G34201,"(33.1011843427186,-115.570115912097)",3.407187,10.747374,0,0,1,0,1,0.526972,New,Same-day,Medium,California,United States,33.101184,-115.570116
4,G69749,"(49.0223887139561,-123.155574774425)",14.276042,3.441740,1,1,1,0,0,0.562618,Repeat,Express,Low,British Columbia,Canada,49.022389,-123.155575


In [7]:
from sklearn.preprocessing import StandardScaler,LabelEncoder,OrdinalEncoder

numerical_cols = ['order_frequency','returns_rate']
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [10]:
selected_features = ['order_frequency', 'returns_rate', 'latitude', 'longitude']
k = 4
kmeans = KMeans(n_clusters = k, verbose = 1)
clusters = kmeans.fit_predict(df[selected_features])
df['clusters'] = clusters


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 3536861.3032486225.
Iteration 1, inertia 2694516.507252563.
Iteration 2, inertia 2659018.4124473473.
Iteration 3, inertia 2640378.425793373.
Iteration 4, inertia 2626903.8950273213.
Iteration 5, inertia 2617037.5611407505.
Iteration 6, inertia 2610844.83291765.
Iteration 7, inertia 2606105.7942992514.
Iteration 8, inertia 2602749.3729879735.
Iteration 9, inertia 2600398.243731428.
Iteration 10, inertia 2598379.503189888.
Iteration 11, inertia 2596515.522334316.
Iteration 12, inertia 2594837.677485389.
Iteration 13, inertia 2592982.926671914.
Iteration 14, inertia 2590690.5404398427.
Iteration 15, inertia 2587875.735767477.
Iteration 16, inertia 2584531.3352559805.
Iteration 17, inertia 2581685.2359950272.
Iteration 18, inertia 2579770.0865899283.
Iteration 19, inertia 2578557.0929992907.
Iteration 20, inertia 2577890.3070798153.
Iteration 21, inertia 2577453.4688456776.
Iteration 22, inertia 2577116.2711596456.
Iteration 23, inertia 2576863.

In [ ]:
cluster_centroids = kmeans.cluster_centers_
cluster_centroids

In [14]:
cluster_centroids = kmeans.cluster_centers_
for cluster_id, (lat, lon, order_freq, returns_rate) in enumerate(cluster_centroids):
    print(f"Cluster {cluster_id} Centroid:")
    print(f"Latitude: {lat}")
    print(f"Longitude: {lon}")
    print(f"Order Frequency: {order_freq}")
    print(f"Returns Rate: {returns_rate}")
    print("\n")

Cluster 0 Centroid:
Latitude: 0.0029880909736349002
Longitude: -0.00794412011178862
Order Frequency: 42.16371907832068
Returns Rate: -114.97452124847368


Cluster 1 Centroid:
Latitude: -0.005330457442526115
Longitude: 0.0017284609357235243
Order Frequency: 41.129360142105575
Returns Rate: -79.35935593268651


Cluster 2 Centroid:
Latitude: 0.00012107954409634283
Longitude: 0.015478445566704154
Order Frequency: 31.48216056533576
Returns Rate: -104.63622126405167


Cluster 3 Centroid:
Latitude: 0.0022452085965025573
Longitude: -0.006147857379815547
Order Frequency: 40.82663907425968
Returns Rate: -94.51646436444484




In [17]:
for i in range(k):
  cluster_center = [cluster_centroids[i][0],cluster_centroids[i][1]]

  geolocator = Nominatim(user_agent="city-extraction")
  location = geolocator.reverse(cluster_center, exactly_one=True)
  address = location.raw.get('address', {})
  location_name = address.get('state', '')

  folium.Marker(
      location = cluster_center,
      icon = folium.Icon(color = 'blue'),
      popup = f"{location_name}"
  ).add_to(m)

m.save('map1.html')
m

AttributeError: ignored